In [ ]:
%matplotlib inline
import cv2
import os,sys
import pickle
import scipy.stats
import sklearn.metrics

import matplotlib
import matplotlib.image as mpimg
import matplotlib.cm as cm # To colour dots of scatter plots
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from skimage.feature import hog
from scipy.stats import norm
from sklearn import preprocessing
from itertools import combinations
from PIL import Image

from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.insert(1, './utilities/')

from utility_functions import *

# Load the Data

In [ ]:
# Loaded a set of images
root_dir = "../../Datasets/training/"

image_dir = root_dir + "images/"
files = os.listdir(image_dir)
n = min(20, len(files)) # Load maximum 20 images
print("Loading " + str(n) + " images")
imgs = [load_image(image_dir + files[i]) for i in range(n)]
print(files[0])

gt_dir = root_dir + "groundtruth/"
print("Loading " + str(n) + " images")
gt_imgs = [load_image(gt_dir + files[i]) for i in range(n)]
print(files[0])

n = 10 # Only use 10 images for training

# Data Preprocessing

## Split the images into smaller patches

In [ ]:
# Extract patches from input images
# size = 10(nb_images)*625(n_patches_per_image)*16*16(size_patch)*3(nb_channels)
patch_size = 16  # each patch is 16*16 pixels

img_patches = [img_crop(imgs[i], patch_size, patch_size) for i in range(n)]
gt_patches = [img_crop(gt_imgs[i], patch_size, patch_size) for i in range(n)]

# Linearize list of patches
img_patches = np.asarray([img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))])
gt_patches =  np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])
#return array of size 6250 containing all the 625 patches for all the 10 training images
#gt has only 0 or 1 values
#img has an rgb value of size 3 for each point

# Extraction of Features from the images

In [ ]:
#TODO: More features + feature interaction

In [ ]:
#line detection
#TODO: transformer ca en feature
def proba_hough_tranform(img):
    img = img_float_to_uint8(img)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    #edges = cv2.Canny(gray,50,150,apertureSize = 3)
    edges = ndimage.gaussian_laplace(gray, sigma=2.5)
    features = np.zeros_like(2*img.shape[0])
    
    #plt.imshow(edges)
    #plt.show()
    minLineLength = 100
    maxLineGap = 120
    lines = cv2.HoughLinesP(edges,0.01,np.pi/360,100,minLineLength,maxLineGap)
    for i in range(len(lines)):
        for x1,y1,x2,y2 in lines[i]:
            cv2.line(img,(x1,y1),(x2,y2),(0,255,0),2)
            
    #cv2.imwrite('houghlines5.jpg',img)
    plt.imshow(img)
    plt.show()
    
    return features
   
proba_hough_tranform(imgs[3])
plt.imshow(gt_imgs[3])

In [ ]:
#important feature detection (corners)
from skimage.feature import corner_harris, corner_subpix, corner_peaks, CENSURE
from skimage.color import rgb2gray

def corner_detection(image):
    image = image.astype('double')
    img_orig = rgb2gray(image)
    detector = CENSURE(min_scale=1,max_scale=3,mode='DoB',non_max_threshold=0.15,line_threshold=10)
    detector.detect(img_orig)

    coords = corner_peaks(corner_harris(img_orig), min_distance=100)
    coords_subpix = corner_subpix(img_orig, coords, window_size=13)

    plt.title('CENSURE feature detection')
    plt.imshow(img_orig)
    plt.scatter(detector.keypoints[:, 1], detector.keypoints[:, 0],
            2 ** detector.scales, facecolors='none', edgecolors='r')
    
corner_detection(imgs[0])
plt.imshow(gt_imgs[0])
#rem: there are a lot of corners along roads

In [ ]:
#example of whole pipeline
img = rgb2gray(imgs[3])
plt.imshow(img)
plt.show()
img = img_float_to_uint8(img)
ret,thresh1 = cv2.threshold(img,127,255,cv2.THRESH_BINARY)
kernel = np.ones((5,5),np.uint8)
erosion = cv2.erode(thresh1,kernel,iterations = 1)
plt.imshow(erosion)
plt.show()
#Removing noise from image
blur = cv2.blur(img,(5,5))
plt.imshow(blur)
plt.show()
#finding edges using edge detection
edges = cv2.Canny(blur, 100 ,200)
plt.imshow(edges)
plt.show()

laplacian = cv2.Laplacian(edges, cv2.CV_8UC1)
sobely = cv2.Sobel(laplacian,cv2.CV_8UC1, 0, 1, ksize=5)
plt.imshow(edges)
plt.show()

# Do a dilation and erosion to accentuate the triangle shape
dilated = cv2.dilate(sobely,kernel,iterations = 1)
#erosion = cv2.erode(dilated,kernel,iterations = 1)

plt.imshow(dilated)
plt.show()
plt.imshow(gt_imgs[3])

In [ ]:
def hog_features(image):
    #histogram of oriented gradients
    grey_bombus = rgb2gray(image)
    hog_features, hog_image = hog(grey_bombus,
                              visualize=True,
                              block_norm='L2-Hys',
                              pixels_per_cell=(8,8))
        
    #plt.imshow(hog_image, cmap=cm.gray)
    return hog_image

In [ ]:
#3 new features
import mahotas

def fd_hu_moments(image):
    image = img_float_to_uint8(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    feature = cv2.HuMoments(cv2.moments(image)).flatten()
    return feature
def fd_haralick(image):    # convert the image to grayscale
    image = img_float_to_uint8(image)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # compute the haralick texture feature vector
    haralick = mahotas.features.haralick(gray).mean(axis=0)
    return haralick
 
#fd hist ne marche pas encore
def fd_histogram(image, mask=None):
    image = img_float_to_uint8(image)
    # convert the image to HSV color-space
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # compute the color histogram
    hist  = cv2.calcHist([image], [0, 1, 2], None, [3, 3, 3], [0, 256, 0, 256, 0, 256])
    # normalize the histogram
    cv2.normalize(hist, hist)
    hist.flatten()
    
print(fd_hu_moments(imgs[3]))
print(fd_haralick(imgs[3]) ) 

In [ ]:
#Features definition
def features_definition(img_patches):
    print("compute X1")
    X1 = np.asarray([ extract_features(img_patches[i]) for i in range(len(img_patches))]) # dim 6
    print("Added dims: " + str(X1.shape[1]))
    X2 = np.asarray([ laplace_gaussian_edge_detection(img_patches[i]) for i in range(len(img_patches))]) # dim 102
    print("Added dims: " + str(X2.shape[1]))
    X3 = np.asarray([ horizontal_and_vertical_edge_detection(img_patches[i]) for i in range(len(img_patches))]) # dim 6
    print("Added dims: " + str(X3.shape[1]))
    X4 = np.asarray([ get_grey_features(get_gray_mask(img_patches[i])) for i in range(len(img_patches))]) # dim 32
    #X4 = np.asarray([ get_gray_mask(img_patches[i]).ravel() for i in range(len(img_patches))]) # dim 16 * 16
    print("Added dims: " + str(X4.shape[1]))
    #X5 = np.asarray([ to_grayscale(img_patches[i]).ravel() for i in range(len(img_patches))]) # dim 16 * 16
    #print("Added dims: " + str(X5.shape[1]))
    X6 = np.asarray([ fd_hu_moments(img_patches[i]) for i in range(len(img_patches))]) # dim 7
    print("Added dims: " + str(X6.shape[1]))
    X7 = np.asarray([ fd_haralick(img_patches[i]) for i in range(len(img_patches))]) # dim 13
    print("Added dims: " + str(X7.shape[1]))
    X8 = np.asarray([ hog_features(img_patches[i]).ravel() for i in range(len(img_patches))]) # dim 16 * 16
    print("Added dims: " + str(X8.shape[1]))
    print("stop")
    X = np.concatenate((X1,X2,X3,X4,X6,X7,X8),axis=1) # dim 402
    #X = feature_expansion(X,4) # dim = dim * degree
    X = add_offset(X) # dim = dim + 1
    print(X.shape)

    return X

In [ ]:
X = features_definition(img_patches)

In [ ]:
#normalization
scaler = preprocessing.MinMaxScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

### Definition of the output y

In [ ]:
### Definition of two classes for the patches

# Compute features for each image patch
foreground_threshold = 0.25 # percentage of pixels > 1 required to assign a foreground label to a patch

def value_to_class(v):
    df = np.sum(v)
    if df > foreground_threshold:
        return 1
    else:
        return 0

#the class foreground has a bit of road in it (25% min)
Y = np.asarray([value_to_class(np.mean(gt_patches[i])) for i in range(len(gt_patches))])

### Statistics on computed features 

In [ ]:
# Print feature statistics
print('Computed ' + str(X.shape[0]) + ' features')
print('Feature dimension = ' + str(X.shape[1]))
print('Number of classes = ' + str(np.max(Y)))  #TODO: fix, length(unique(Y)) 

Y0 = [i for i, j in enumerate(Y) if j == 0]
Y1 = [i for i, j in enumerate(Y) if j == 1]
print('Class 0: ' + str(len(Y0)) + ' samples')
print('Class 1: ' + str(len(Y1)) + ' samples')

In [ ]:
def hist_of_features(X):
    """
    performs a histogram of the distribution of all the parameters in the dataset 
    parameters:
        X: the dataset to plot
    """
    n = X.shape[1]
    fig, ax = plt.subplots(4,5)

    for i in range(4):
        for j in range(5):
            ax[i,j].hist(X[:,i+j], bins = 30)
            ax[i,j].set_title(i+j,fontsize=40)
        
    fig.set_figheight(150)
    fig.set_figwidth(150)
    plt.suptitle("Distribution of all the features",fontsize=150)
    plt.tight_layout()
    fig.subplots_adjust(top=0.92)
    plt.show()
    
hist_of_features(X)

## PCA and Plots

In [ ]:
pca = PCA(n_components=10)
pca.fit(X.T)  
colors = ['c' if yi else 'm' for yi in Y]
print(pca.explained_variance_ratio_[:10], pca.components_.shape)  
plt.scatter(pca.components_[0],Y, s=20, c=colors)
#first component explains 99% variance -> but still no clear separation between y1 and y0
    
plt.show()

In [ ]:
plt.scatter(np.log(pca.components_[0]),np.log(pca.components_[2]), s=10, c=colors)
print(np.mean(pca.components_[0]), np.var(pca.components_[0]))
print(np.where(pca.components_[1] > 0.1), pca.components_[0].shape)

# Training of the model

In [ ]:
#define parameters
proportion_class0 = (len(Y0)) / (len(Y))
proportion_class1 = (len(Y1)) / (len(Y))
print(proportion_class0,proportion_class1)
weight = {0:proportion_class1, 1:proportion_class0}

## 1) Logistic regression

In [ ]:
# train a logistic regression classifier
from sklearn import linear_model
import time

# we create an instance of the classifier and fit the data
logreg = linear_model.LogisticRegression(C=1e4,solver='liblinear',penalty='l1',class_weight=weight,verbose=3)
# Run the estimation
start = time.time()
logreg.fit(pca.components_.T, Y)
stop = time.time()
print(stop - start)

In [ ]:
print(X.shape)

In [ ]:
# Predict on the training set
Z = logreg.predict(pca.components_.T)

In [ ]:
#Best score yet: 0.230769

## 2) SVM

In [ ]:
#in general the SVM performs better than the LogReg model
from sklearn import svm
#pca.components_.T

clf = svm.SVC(C=1.0, cache_size=200, class_weight=weight, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=True)
clf.fit(X, Y)  

In [ ]:
# Predict on the training set
Z = clf.predict(X)

In [ ]:
print(sum(Z),sum(Y))

## Model evaluation

In [ ]:
# Get non-zeros in prediction and grountruth arrays
Zn = np.nonzero(Z)[0]
Yn = np.nonzero(Y)[0]
recall = len(list(set(Yn) & set(Zn))) / float(len(Z)) #Recall = TPR
precision = len(list(set(Yn) & set(Zn))) / float(len(Y))
F1 = 2*(recall*precision)/(recall+precision)
print('F1 Score = ' + str(F1))

# Display of results

In [ ]:
img_idx = 12

Xi = extract_img_features(image_dir + files[img_idx],patch_size)
Zi = logreg.predict(Xi)
plt.scatter(Xi[:, 1], Xi[:, 10], c=Zi, edgecolors='k', cmap=plt.cm.Paired)

In [ ]:
w = gt_imgs[img_idx].shape[0]
h = gt_imgs[img_idx].shape[1]
predicted_im = label_to_img(w, h, patch_size, patch_size, Zi)
cimg = concatenate_images(imgs[img_idx], predicted_im)
fig1 = plt.figure(figsize=(10, 10)) # create a figure with the default size 
plt.imshow(cimg, cmap='Greys_r')

new_img = make_img_overlay(imgs[img_idx], predicted_im)

plt.imshow(new_img)